# New Homological Codes - Comprehensive Test

This notebook provides systematic testing of new/advanced homological codes:
1. **Setup** - Module cache clearing, imports
2. **Code Import Tests** - Test import availability of new code types
3. **Code Instantiation Grid** - Test construction of all new codes
4. **CSS Property Validation** - Verify CSS orthogonality conditions
5. **Summary** - Pass/fail statistics

Reference implementation from decoder_smoke_test.ipynb patterns.

In [ ]:
"""Setup - New Homological Codes Testing - Using testing_utils"""
import sys
import time
import warnings
import numpy as np

warnings.filterwarnings('ignore')

# Clear module cache for fresh testing
from qectostim.testing import clear_qectostim_modules
clear_qectostim_modules()

# Import testing utilities
from qectostim.testing import (
    test_code_circuit,
    test_composite_construction,
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
)

print("✓ Module cache cleared using testing_utils")
print("✓ test_code_circuit and test_composite_construction imported")
print("Ready for new homological codes testing")

✓ Module cache cleared
Ready for new homological codes testing


In [2]:
"""Test 1: Import Availability of New Codes"""
print("Import Availability Tests:")
print("=" * 70)

import_results = []

# Test code import categories
import_tests = [
    # (name, import_statement)
    ("3D/4D Toric", "from qectostim.codes.surface import ToricCode3D, ToricCode3DFaces, ToricCode4D"),
    ("3D Color", "from qectostim.codes.color import ColorCode3D, ColorCode3DPrism"),
    ("Fracton", "from qectostim.codes.topological import XCubeCode, HaahCode, CheckerboardCode"),
    ("QLDPC Base", "from qectostim.codes.qldpc import TillichZemorHGP, HomologicalProductCode"),
    ("Composite", "from qectostim.codes.composite import ConcatenatedCSSCode, DualCode"),
    ("Floquet", "from qectostim.codes.floquet import FloquetHoneycombCode"),
    ("Subsystem", "from qectostim.codes.subsystem import BaconShorCode, GaugeColorCode"),
    ("Small Codes", "from qectostim.codes.small import FourQubit422Code, SteanCode713, ShorCode"),
    ("Surface Codes", "from qectostim.codes.surface import RotatedSurfaceCode, PlanarSurfaceCode"),
]

for name, import_stmt in import_tests:
    try:
        exec(import_stmt)
        import_results.append((name, "✓"))
        print(f"✓ {name}")
    except ImportError as e:
        import_results.append((name, f"✗ ImportError: {str(e)[:40]}"))
        print(f"✗ {name}: ImportError - {str(e)[:50]}")
    except Exception as e:
        import_results.append((name, f"✗ {type(e).__name__}"))
        print(f"✗ {name}: {type(e).__name__} - {str(e)[:50]}")

passed = sum(1 for r in import_results if r[1] == "✓")
print(f"\nImport Tests: {passed}/{len(import_results)} passed")

Import Availability Tests:
✓ 3D/4D Toric
✓ 3D Color
✓ Fracton
✗ QLDPC Base: ImportError - cannot import name 'TillichZemorHGP' from 'qectost
✓ Composite
✗ Floquet: ImportError - cannot import name 'FloquetHoneycombCode' from 'qe
✓ Subsystem
✗ Small Codes: ImportError - cannot import name 'ShorCode' from 'qectostim.code
✗ Surface Codes: ImportError - cannot import name 'PlanarSurfaceCode' from 'qecto

Import Tests: 5/9 passed


In [3]:
"""Test 2: Code Instantiation Grid"""
print("Code Instantiation Tests:")
print("=" * 70)

instantiation_results = []

# Define codes to test with their constructors
codes_to_test = []

# Small Codes (always available)
try:
    from qectostim.codes.small.four_two_two import FourQubit422Code
    codes_to_test.append(("FourQubit422", lambda: FourQubit422Code()))
except: pass

try:
    from qectostim.codes.small.steane_713 import SteanCode713
    codes_to_test.append(("SteanCode713", lambda: SteanCode713()))
except: pass

try:
    from qectostim.codes.small.shor_91 import ShorCode
    codes_to_test.append(("ShorCode", lambda: ShorCode()))
except: pass

# Surface Codes
try:
    from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
    codes_to_test.append(("RotatedSurface(d=3)", lambda: RotatedSurfaceCode(distance=3)))
except: pass

try:
    from qectostim.codes.surface.planar_surface import PlanarSurfaceCode
    codes_to_test.append(("PlanarSurface(d=3)", lambda: PlanarSurfaceCode(distance=3)))
except: pass

# Composite Codes
try:
    from qectostim.codes.composite.concatenated import ConcatenatedCSSCode
    from qectostim.codes.small.four_two_two import FourQubit422Code
    from qectostim.codes.small.steane_713 import SteanCode713
    codes_to_test.append(("ConcatenatedCSS(422,Steane)", lambda: ConcatenatedCSSCode(SteanCode713(), FourQubit422Code())))
except: pass

try:
    from qectostim.codes.composite.dual import DualCode
    from qectostim.codes.small.steane_713 import SteanCode713
    codes_to_test.append(("DualCode(Steane)", lambda: DualCode(SteanCode713())))
except: pass

# QLDPC
try:
    from qectostim.codes.qldpc.tillich_zemor import TillichZemorHGP
    codes_to_test.append(("TillichZemorHGP(n=7)", lambda: TillichZemorHGP(n=7)))
except: pass

try:
    from qectostim.codes.qldpc.homological_product import HomologicalProductCode
    from qectostim.codes.small.steane_713 import SteanCode713
    codes_to_test.append(("HomologicalProduct(Steane,Steane)", lambda: HomologicalProductCode(SteanCode713(), SteanCode713())))
except: pass

print(f"Testing {len(codes_to_test)} code types...")
print(f"{'Name':<40} {'n':>5} {'k':>5} {'Status':<25}")
print("-" * 80)

for name, constructor in codes_to_test:
    try:
        start = time.time()
        code = constructor()
        elapsed = time.time() - start
        
        n = getattr(code, 'n', '?')
        k = getattr(code, 'k', '?')
        
        instantiation_results.append((name, n, k, f"✓ ({elapsed:.3f}s)"))
        print(f"{name:<40} {n:>5} {k:>5} {'✓':>10} ({elapsed:.3f}s)")
        
    except Exception as e:
        instantiation_results.append((name, '?', '?', f"✗ {type(e).__name__}"))
        print(f"{name:<40} {'?':>5} {'?':>5} {'✗ ' + type(e).__name__[:15]:<25}")

passed = sum(1 for r in instantiation_results if r[3].startswith('✓'))
print(f"\nInstantiation: {passed}/{len(instantiation_results)} passed")

Code Instantiation Tests:
Testing 5 code types...
Name                                         n     k Status                   
--------------------------------------------------------------------------------
FourQubit422                                 4     2          ✓ (0.000s)
SteanCode713                                 7     1          ✓ (0.000s)
RotatedSurface(d=3)                          9     1          ✓ (0.000s)
ConcatenatedCSS(422,Steane)                 28     8          ✓ (0.000s)
DualCode(Steane)                             7     1          ✓ (0.000s)

Instantiation: 5/5 passed


In [4]:
"""Test 3: CSS Property Validation"""
print("CSS Property Validation:")
print("=" * 70)

css_validation_results = []

# Re-instantiate codes that passed and check CSS properties
for name, n, k, status in instantiation_results:
    if not status.startswith('✓'):
        continue
        
    # Find the constructor
    constructor = None
    for test_name, test_constructor in codes_to_test:
        if test_name == name:
            constructor = test_constructor
            break
    
    if constructor is None:
        continue
    
    try:
        code = constructor()
        
        # Check if CSS code (has hx, hz)
        if not (hasattr(code, 'hx') and hasattr(code, 'hz')):
            css_validation_results.append((name, 'N/A', 'Non-CSS code'))
            print(f"  {name}: Non-CSS (skipped)")
            continue
        
        hx, hz = code.hx, code.hz
        
        # Check CSS orthogonality: hx @ hz.T = 0 (mod 2)
        if hx.size > 0 and hz.size > 0:
            product = (hx @ hz.T) % 2
            orthogonal = np.all(product == 0)
        else:
            orthogonal = True  # Empty matrices are trivially orthogonal
        
        if orthogonal:
            css_validation_results.append((name, '✓', 'CSS orthogonality satisfied'))
            print(f"✓ {name}: CSS orthogonality ✓")
        else:
            css_validation_results.append((name, '✗', 'CSS orthogonality violated'))
            print(f"✗ {name}: CSS orthogonality violated")
            
    except Exception as e:
        css_validation_results.append((name, '✗', f'{type(e).__name__}'))
        print(f"✗ {name}: {type(e).__name__}: {str(e)[:40]}")

passed = sum(1 for r in css_validation_results if r[1] == '✓')
total = sum(1 for r in css_validation_results if r[1] != 'N/A')
print(f"\nCSS Validation: {passed}/{total} passed")

CSS Property Validation:
✓ FourQubit422: CSS orthogonality ✓
✓ SteanCode713: CSS orthogonality ✓
✓ RotatedSurface(d=3): CSS orthogonality ✓
✓ ConcatenatedCSS(422,Steane): CSS orthogonality ✓
✓ DualCode(Steane): CSS orthogonality ✓

CSS Validation: 5/5 passed


In [5]:
"""Test 4: Stim Circuit Generation from New Codes"""
print("Stim Circuit Generation:")
print("=" * 70)

from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

circuit_results = []

# Test circuit generation for a few codes
circuit_test_codes = [
    ("RotatedSurface(d=3)", lambda: RotatedSurfaceCode(distance=3)),
    ("FourQubit422", lambda: FourQubit422Code()),
    ("SteanCode713", lambda: SteanCode713()),
]

print(f"{'Name':<30} {'Qubits':>8} {'Detectors':>10} {'Status':<20}")
print("-" * 70)

for name, constructor in circuit_test_codes:
    try:
        code = constructor()
        noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)
        exp = CSSMemoryExperiment(code=code, rounds=2, noise_model=noise)
        
        circuit = exp.to_stim()
        num_qubits = circuit.num_qubits
        num_detectors = circuit.num_detectors
        
        circuit_results.append((name, num_qubits, num_detectors, '✓'))
        print(f"{name:<30} {num_qubits:>8} {num_detectors:>10} {'✓':<20}")
        
    except Exception as e:
        circuit_results.append((name, '-', '-', f'✗ {type(e).__name__}'))
        print(f"{name:<30} {'-':>8} {'-':>10} {'✗ ' + type(e).__name__[:15]:<20}")

passed = sum(1 for r in circuit_results if r[3] == '✓')
print(f"\nCircuit Generation: {passed}/{len(circuit_results)} passed")

Stim Circuit Generation:
Name                             Qubits  Detectors Status              
----------------------------------------------------------------------
RotatedSurface(d=3)                  17         20 ✓                   
FourQubit422                          6          5 ✓                   
SteanCode713                         13         15 ✓                   

Circuit Generation: 3/3 passed


In [6]:
"""Summary: New Codes Test Results"""
print("=" * 70)
print("NEW HOMOLOGICAL CODES TEST SUMMARY")
print("=" * 70)

# Aggregate results
summary = {
    "Import Tests": sum(1 for r in import_results if r[1] == "✓"),
    "Instantiation": sum(1 for r in instantiation_results if r[3].startswith('✓')),
    "CSS Validation": sum(1 for r in css_validation_results if r[1] == '✓'),
    "Circuit Generation": sum(1 for r in circuit_results if r[3] == '✓'),
}

totals = {
    "Import Tests": len(import_results),
    "Instantiation": len(instantiation_results),
    "CSS Validation": sum(1 for r in css_validation_results if r[1] != 'N/A'),
    "Circuit Generation": len(circuit_results),
}

total_passed = sum(summary.values())
total_tests = sum(totals.values())

print(f"\nTest Results:")
for test_name in summary:
    passed = summary[test_name]
    total = totals[test_name]
    print(f"  {test_name}: {passed}/{total} passed")

print(f"\nOverall: {total_passed}/{total_tests} tests passed")

# Show any failures
failures = []
for r in import_results:
    if r[1] != "✓":
        failures.append(('Import', r[0], r[1]))
for r in instantiation_results:
    if not r[3].startswith('✓'):
        failures.append(('Instantiation', r[0], r[3]))
for r in css_validation_results:
    if r[1] == '✗':
        failures.append(('CSS', r[0], r[2]))
for r in circuit_results:
    if r[3] != '✓':
        failures.append(('Circuit', r[0], r[3]))

if failures:
    print(f"\nFailures ({len(failures)}):")
    for cat, name, err in failures[:10]:
        print(f"  [{cat}] {name}: {err}")
else:
    print("\n✓ ALL TESTS PASSED")

NEW HOMOLOGICAL CODES TEST SUMMARY

Test Results:
  Import Tests: 5/9 passed
  Instantiation: 5/5 passed
  CSS Validation: 5/5 passed
  Circuit Generation: 3/3 passed

Overall: 18/22 tests passed

Failures (4):
  [Import] QLDPC Base: ✗ ImportError: cannot import name 'TillichZemorHGP' fro
  [Import] Floquet: ✗ ImportError: cannot import name 'FloquetHoneycombCode
  [Import] Small Codes: ✗ ImportError: cannot import name 'ShorCode' from 'qect
  [Import] Surface Codes: ✗ ImportError: cannot import name 'PlanarSurfaceCode' f
